In [4]:
# Notebook Block 1: Importing Libraries
import os
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import concurrent.futures
import tensorflow as tf
from tqdm import tqdm

# Step 1: Data Preprocessing
def download_image(args):
    url, image_folder, category, catalog = args
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        folder_path = os.path.join(image_folder, catalog, category)
        os.makedirs(folder_path, exist_ok=True)
        filename = os.path.join(folder_path, os.path.basename(url))
        img.save(filename)
        return filename
    except:
        return None

def preprocess_image(image_path, target_size=(224, 224)):
    img = Image.open(image_path)
    img = img.resize(target_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    return img_array / 255.0

def preprocess_dataset(dataset_path, image_folder):
    df = pd.read_csv(dataset_path)
    
    # Download images in parallel with ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        args_list = [(url, image_folder, category, catalog) for url, category, catalog in zip(df["Image Links"], df["Category Names(readonly)"], df["Catalog Names(readonly)"])]
        with tqdm(total=len(args_list), desc="Downloading Images") as pbar:
            image_paths = list(executor.map(download_image, args_list))
            pbar.update(len(image_paths))
    
    df["Image Paths"] = image_paths
    df = df.dropna(subset=["Image Paths"])  # Drop rows with missing image paths
    df["Category"] = pd.Categorical(df["Category Names(readonly)"]).codes
    df["Catalog"] = pd.Categorical(df["Catalog Names(readonly)"]).codes
    
    # Save image labels to a file using 'utf-8' encoding
    with open('image_labels.txt', 'w', encoding='utf-8') as f:
        for idx, row in df.iterrows():
            image_path = row["Image Paths"]
            category = row["Category Names(readonly)"]
            catalog = row["Catalog Names(readonly)"]
            f.write(f"{image_path}\t{category}\t{catalog}\n")
    
    return df

# Configuration
dataset_path = r'D:\PENNY\Gaiat products.csv'
image_folder = r'D:\PENNY\Gaiat products\images'
num_categories = 82  # Fill in the number of unique categories in your dataset
num_catalogs = 18  # Fill in the number of unique catalogs in your dataset

# Step 1: Data Preprocessing
df = preprocess_dataset(dataset_path, image_folder)

# Create a TensorFlow Dataset for efficient data loading and preprocessing
image_paths = df["Image Paths"].tolist()
categories = df["Category"].tolist()

AUTOTUNE = tf.data.AUTOTUNE

def load_and_preprocess_image(image_path, category):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224))
    img = tf.cast(img, tf.float32) / 255.0
    return img, category

dataset = tf.data.Dataset.from_tensor_slices((image_paths, categories))
dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)


In [16]:
# Notebook Block 1: Importing Libraries

import os
import pandas as pd
import requests
from PIL import Image, ImageOps, ImageEnhance
from io import BytesIO
import concurrent.futures
import tensorflow as tf
from tqdm import tqdm

# Step 1: Data Preprocessing
def download_image(args):
    url, image_folder, category, catalog = args
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        folder_path = os.path.join(image_folder, catalog, category)
        os.makedirs(folder_path, exist_ok=True)
        filename = os.path.join(folder_path, os.path.basename(url))
        img.save(filename)
        return filename
    except:
        return None

def preprocess_image(image_path, target_size=(224, 224), output_folder=None):
    img = Image.open(image_path)
    img = img.transpose(Image.FLIP_LEFT_RIGHT)  # Reverse the image
    img = ImageOps.autocontrast(img)  # Normalize the image
    enhancer = ImageEnhance.Sharpness(img)
    img = enhancer.enhance(2.0)  # Increase sharpness
    img = img.filter(ImageFilter.SHARPEN)  # Edge sharpening

    if output_folder:
        os.makedirs(output_folder, exist_ok=True)
        filename = os.path.join(output_folder, os.path.basename(image_path))
        img.save(filename)
    else:
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        return img_array / 255.0

def preprocess_catalog(catalog_folder, image_folder):
    category_folders = os.listdir(catalog_folder)
    for category in category_folders:
        category_folder = os.path.join(catalog_folder, category)
        output_folder = os.path.join(image_folder, catalog, category + "_processed")
        os.makedirs(output_folder, exist_ok=True)
        image_files = os.listdir(category_folder)
        with concurrent.futures.ThreadPoolExecutor() as executor:
            args_list = [(os.path.join(category_folder, image_file), (224, 224), output_folder) for image_file in image_files]
            list(tqdm(executor.map(preprocess_image, args_list), total=len(args_list), desc=f"Processing Catalog {os.path.basename(catalog_folder)}"))

def preprocess_dataset(dataset_path, image_folder):
    df = pd.read_csv(dataset_path)
    
    # Download images in parallel with ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        args_list = [(url, image_folder, category, catalog) for url, category, catalog in zip(df["Image Links"], df["Category Names(readonly)"], df["Catalog Names(readonly)"])]
        image_paths = list(tqdm(executor.map(download_image, args_list), total=len(args_list), desc="Downloading Images"))
    
    df["Image Paths"] = image_paths
    df = df.dropna(subset=["Image Paths"])  # Drop rows with missing image paths
    df["Category"] = pd.Categorical(df["Category Names(readonly)"]).codes
    df["Catalog"] = pd.Categorical(df["Catalog Names(readonly)"]).codes
    
    # Save image labels to a file using 'utf-8' encoding
    with open('image_labels.txt', 'w', encoding='utf-8') as f:
        for idx, row in df.iterrows():
            image_path = row["Image Paths"]
            category = row["Category Names(readonly)"]
            catalog = row["Catalog Names(readonly)"]
            f.write(f"{image_path}\t{category}\t{catalog}\n")
    
    # Preprocess each catalog
    catalog_folders = os.listdir(image_folder)
    for catalog in catalog_folders:
        catalog_folder = os.path.join(image_folder, catalog)
        preprocess_catalog(catalog_folder, image_folder)

# Configuration
dataset_path = r'D:\PENNY\Gaiat products.csv'
image_folder = r'D:\PENNY\Gaiat products\images'  # Update to the correct image folder path
num_categories = 82  # Fill in the number of unique categories in your dataset
num_catalogs = 18  # Fill in the number of unique catalogs in your dataset

# Step 1: Data Preprocessing
df = preprocess_dataset(dataset_path, image_folder)

# Create a TensorFlow Dataset for efficient data loading and preprocessing
image_paths = df["Image Paths"].tolist()
categories = df["Category"].tolist()

AUTOTUNE = tf.data.AUTOTUNE

def load_and_preprocess_image(image_path, category):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224))
    img = tf.cast(img, tf.float32) / 255.0
    return img, category

dataset = tf.data.Dataset.from_tensor_slices((image_paths, categories))
dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)


Processing Catalog Cleaning & Disinfections Chemicals:   0%|          | 0/37 [00:00<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'read'

: 

In [7]:
import os
import cv2
import numpy as np
from tqdm import tqdm  # Import tqdm for the progress bar


In [8]:
def normalize_image(image):
    return cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

def reverse_image(image):
    return cv2.bitwise_not(image)

def sharpen_edges(image):
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    return cv2.filter2D(image, -1, kernel)

def enhance_image(image):
    # Apply any other enhancement techniques here as needed.
    # For example, histogram equalization, contrast adjustment, etc.
    return image


In [11]:
import os
import cv2
import numpy as np
from tqdm import tqdm

def normalize_image(image):
    return cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

def reverse_image(image):
    return cv2.bitwise_not(image)

def sharpen_edges(image):
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    try:
        return cv2.filter2D(image, -1, kernel)
    except cv2.error as e:
        print(f"Error while applying sharpen_edges: {e}")
        return image

def enhance_image(image):
    # Apply any other enhancement techniques here as needed.
    # For example, histogram equalization, contrast adjustment, etc.
    return image

input_dir = r'D:\PENNY\Gaiat products\images'

for catalog in os.listdir(input_dir):
    catalog_path = os.path.join(input_dir, catalog)
    if not os.path.isdir(catalog_path):
        continue

    for category in os.listdir(catalog_path):
        category_path = os.path.join(catalog_path, category)
        if not os.path.isdir(category_path):
            continue

        # Use tqdm to create a progress bar for the product images in the category
        product_files = [f for f in os.listdir(category_path) if f.endswith('.jpg')]
        progress_bar = tqdm(total=len(product_files), desc=f"Processing {catalog}/{category}", unit="image")

        for product_file in product_files:
            product_path = os.path.join(category_path, product_file)

            # Check if the image is read correctly
            image = cv2.imread(product_path)
            if image is None:
                print(f"Error: Unable to read image {product_file}")
                continue

            # Apply image processing functions
            processed_image = normalize_image(image)
            processed_image = reverse_image(processed_image)
            processed_image = sharpen_edges(processed_image)
            processed_image = enhance_image(processed_image)

            # Save the processed image in the same directory without deleting the original image
            processed_path = os.path.join(category_path, f"processed_{product_file}")
            cv2.imwrite(processed_path, processed_image)

            progress_bar.update(1)  # Update the progress bar after processing each image

        progress_bar.close()  # Close the progress bar for the current category


Processing Office items/Office stationery:  75%|███████▌  | 3/4 [02:47<00:55, 55.97s/image]


Processing Cleaning & Disinfections Chemicals/Air Fresheners: 100%|██████████| 16/16 [00:04<00:00,  3.55image/s]
Processing Cleaning & Disinfections Chemicals/Disinfections & General Cleaners: 100%|██████████| 22/22 [00:01<00:00, 15.44image/s]
Processing Cleaning & Disinfections Chemicals/Drain Opener: 100%|██████████| 24/24 [00:03<00:00,  7.25image/s]
Processing Cleaning & Disinfections Chemicals/Glass Cleaner: 100%|██████████| 6/6 [00:00<00:00, 13.13image/s]
Processing Cleaning & Disinfections Chemicals/Hand Soap: 100%|██████████| 16/16 [00:02<00:00,  7.21image/s]
Processing Cleaning & Disinfections Chemicals/Kitchen Cleaning Chemicals: 100%|██████████| 10/10 [00:00<00:00, 14.70image/s]
Processing Cleaning & Disinfections Chemicals/Specialized Chemicals: 100%|██████████| 50/50 [00:08<00:00,  6.22image/s]
Processing Cleaning & Disinfections Chemicals/Surface & Floor Cleaning: 100%|██████████| 10/10 [00:01<00:00,  9.67image/s]
Processing Cleaning & Disinfections Chemicals/Toilet Cleani

Error: Unable to read image حامل_ادوات_مكتبية.jpg


Processing Office items/Printers - Desktop: 100%|██████████| 4/4 [00:00<00:00,  5.87image/s]
Processing Office items/Software: 0image [00:00, ?image/s]
Processing Packaging Materials/Aluminum Containers: 0image [00:00, ?image/s]
Processing Packaging Materials/Paper Containers: 0image [00:00, ?image/s]
Processing Packaging Materials/Plastic Containers: 0image [00:00, ?image/s]
Processing Plastic, Tissues, & Paper Wares/Bags & Trash Bags: 100%|██████████| 7/7 [00:01<00:00,  4.91image/s]
Processing Plastic, Tissues, & Paper Wares/Paper Bags: 100%|██████████| 9/9 [00:01<00:00,  4.65image/s]
Processing Plastic, Tissues, & Paper Wares/Paper cups: 100%|██████████| 8/8 [00:01<00:00,  6.82image/s]
Processing Plastic, Tissues, & Paper Wares/Plastic Cups: 100%|██████████| 6/6 [00:00<00:00,  8.66image/s]
Processing Plastic, Tissues, & Paper Wares/Plastic Wares: 100%|██████████| 24/24 [00:05<00:00,  4.52image/s]
Processing Plastic, Tissues, & Paper Wares/Tissues: 100%|██████████| 15/15 [00:03<00:00

In [14]:
for catalog in os.listdir(input_dir):
    catalog_path = os.path.join(input_dir, catalog)
    if not os.path.isdir(catalog_path):
        continue

    for category in os.listdir(catalog_path):
        category_path = os.path.join(catalog_path, category)
        if not os.path.isdir(category_path):
            continue

        # Use tqdm to create a progress bar for the product images in the category
        product_files = [f for f in os.listdir(category_path) if f.endswith('.jpg')]
        progress_bar = tqdm(total=len(product_files), desc=f"Processing {catalog}/{category}", unit="image")

        for product_file in product_files:
            product_path = os.path.join(category_path, product_file)

            # Check if the image is read correctly
            image = cv2.imread(product_path)
            if image is None:
                print(f"Error: Unable to read image {product_file}")
                continue

            # Apply image processing functions
            processed_image = reverse_image(processed_image)

            # Save the processed image in the same directory without deleting the original image
            processed_path = os.path.join(category_path, f"processed_{product_file}")
            cv2.imwrite(processed_path, processed_image)

            progress_bar.update(1)  # Update the progress bar after processing each image

        progress_bar.close()  # Close the progress bar for the current category


Processing Cleaning & Disinfections Chemicals/Air Fresheners:   0%|          | 0/32 [00:00<?, ?image/s]

Processing Cleaning & Disinfections Chemicals/Air Fresheners: 100%|██████████| 32/32 [00:10<00:00,  3.12image/s]
Processing Cleaning & Disinfections Chemicals/Disinfections & General Cleaners: 100%|██████████| 44/44 [00:16<00:00,  2.73image/s]
Processing Cleaning & Disinfections Chemicals/Drain Opener: 100%|██████████| 48/48 [00:19<00:00,  2.50image/s]
Processing Cleaning & Disinfections Chemicals/Glass Cleaner: 100%|██████████| 12/12 [00:03<00:00,  3.04image/s]
Processing Cleaning & Disinfections Chemicals/Hand Soap: 100%|██████████| 32/32 [00:10<00:00,  3.19image/s]
Processing Cleaning & Disinfections Chemicals/Kitchen Cleaning Chemicals: 100%|██████████| 20/20 [00:07<00:00,  2.65image/s]
Processing Cleaning & Disinfections Chemicals/Specialized Chemicals: 100%|██████████| 100/100 [00:41<00:00,  2.40image/s]
Processing Cleaning & Disinfections Chemicals/Surface & Floor Cleaning: 100%|██████████| 20/20 [00:11<00:00,  1.73image/s]
Processing Cleaning & Disinfections Chemicals/Toilet Cl

Error: Unable to read image حامل_ادوات_مكتبية.jpg


Processing Office items/Printers - Desktop: 100%|██████████| 12/12 [00:03<00:00,  3.30image/s]
Processing Office items/Software: 0image [00:00, ?image/s]
Processing Packaging Materials/Aluminum Containers: 0image [00:00, ?image/s]
Processing Packaging Materials/Paper Containers: 0image [00:00, ?image/s]
Processing Packaging Materials/Plastic Containers: 0image [00:00, ?image/s]
Processing Plastic, Tissues, & Paper Wares/Bags & Trash Bags: 100%|██████████| 21/21 [00:06<00:00,  3.28image/s]
Processing Plastic, Tissues, & Paper Wares/Paper Bags: 100%|██████████| 27/27 [00:09<00:00,  2.87image/s]
Processing Plastic, Tissues, & Paper Wares/Paper cups: 100%|██████████| 24/24 [00:08<00:00,  2.93image/s]
Processing Plastic, Tissues, & Paper Wares/Plastic Cups: 100%|██████████| 18/18 [00:05<00:00,  3.12image/s]
Processing Plastic, Tissues, & Paper Wares/Plastic Wares: 100%|██████████| 72/72 [00:38<00:00,  1.87image/s]
Processing Plastic, Tissues, & Paper Wares/Tissues: 100%|██████████| 45/45 [0

In [15]:
import os
import cv2
import numpy as np
from tqdm import tqdm

def normalize_image(image):
    return cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

def reverse_image(image):
    return cv2.bitwise_not(image)

def sharpen_edges(image):
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    return cv2.filter2D(image, -1, kernel)

def enhance_image(image):
    # Apply any other enhancement techniques here as needed.
    # For example, histogram equalization, contrast adjustment, etc.
    return image

def save_processed_image(image, output_path):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, image)

input_dir = r'D:\PENNY\Gaiat products\images'

for catalog in os.listdir(input_dir):
    catalog_path = os.path.join(input_dir, catalog)
    if not os.path.isdir(catalog_path):
        continue

    for category in os.listdir(catalog_path):
        category_path = os.path.join(catalog_path, category)
        if not os.path.isdir(category_path):
            continue

        # Use tqdm to create a progress bar for the product images in the category
        product_files = [f for f in os.listdir(category_path) if f.endswith('.jpg')]
        progress_bar = tqdm(total=len(product_files), desc=f"Processing {catalog}/{category}", unit="image")

        for product_file in product_files:
            product_path = os.path.join(category_path, product_file)

            # Check if the image is read correctly
            image = cv2.imread(product_path)
            if image is None:
                print(f"Error: Unable to read image {product_file}")
                continue

            # Apply image processing functions
            processed_image = normalize_image(image)
            processed_image = reverse_image(processed_image)
            processed_image = sharpen_edges(processed_image)
            processed_image = enhance_image(processed_image)

            # Save the processed images as individual files
            processed_base_name = os.path.splitext(product_file)[0]
            save_processed_image(processed_image, os.path.join(category_path, f"{processed_base_name}_normalized.jpg"))
            save_processed_image(reverse_image(image), os.path.join(category_path, f"{processed_base_name}_reversed.jpg"))
            save_processed_image(sharpen_edges(image), os.path.join(category_path, f"{processed_base_name}_sharpened.jpg"))
            save_processed_image(enhance_image(image), os.path.join(category_path, f"{processed_base_name}_enhanced.jpg"))

            progress_bar.update(1)  # Update the progress bar after processing each image

        progress_bar.close()  # Close the progress bar for the current category


Processing Cleaning & Disinfections Chemicals/Air Fresheners: 100%|██████████| 40/40 [00:18<00:00,  2.21image/s]
Processing Cleaning & Disinfections Chemicals/Disinfections & General Cleaners: 100%|██████████| 55/55 [00:44<00:00,  1.24image/s]
Processing Cleaning & Disinfections Chemicals/Drain Opener: 100%|██████████| 60/60 [00:30<00:00,  2.00image/s]
Processing Cleaning & Disinfections Chemicals/Glass Cleaner: 100%|██████████| 15/15 [00:05<00:00,  2.92image/s]
Processing Cleaning & Disinfections Chemicals/Hand Soap: 100%|██████████| 40/40 [00:15<00:00,  2.61image/s]
Processing Cleaning & Disinfections Chemicals/Kitchen Cleaning Chemicals: 100%|██████████| 25/25 [00:09<00:00,  2.50image/s]
Processing Cleaning & Disinfections Chemicals/Specialized Chemicals: 100%|██████████| 125/125 [00:54<00:00,  2.31image/s]
Processing Cleaning & Disinfections Chemicals/Surface & Floor Cleaning: 100%|██████████| 25/25 [00:09<00:00,  2.66image/s]
Processing Cleaning & Disinfections Chemicals/Toilet Cl

Error: Unable to read image حامل_ادوات_مكتبية.jpg


Processing Office items/Printers - Desktop: 100%|██████████| 16/16 [00:07<00:00,  2.28image/s]
Processing Office items/Software: 0image [00:00, ?image/s]
Processing Packaging Materials/Aluminum Containers: 0image [00:00, ?image/s]
Processing Packaging Materials/Paper Containers: 0image [00:00, ?image/s]
Processing Packaging Materials/Plastic Containers: 0image [00:00, ?image/s]
Processing Plastic, Tissues, & Paper Wares/Bags & Trash Bags: 100%|██████████| 28/28 [00:14<00:00,  1.87image/s]
Processing Plastic, Tissues, & Paper Wares/Paper Bags: 100%|██████████| 36/36 [00:20<00:00,  1.77image/s]
Processing Plastic, Tissues, & Paper Wares/Paper cups: 100%|██████████| 32/32 [00:12<00:00,  2.61image/s]
Processing Plastic, Tissues, & Paper Wares/Plastic Cups: 100%|██████████| 24/24 [00:09<00:00,  2.64image/s]
Processing Plastic, Tissues, & Paper Wares/Plastic Wares: 100%|██████████| 96/96 [00:37<00:00,  2.54image/s]
Processing Plastic, Tissues, & Paper Wares/Tissues: 100%|██████████| 60/60 [0

In [16]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

def normalize_image(image):
    return cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

def reverse_image(image):
    return cv2.bitwise_not(image)

def sharpen_edges(image):
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    return cv2.filter2D(image, -1, kernel)

def resize_image(image, new_size=(200, 200)):
    return cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)

def process_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to read image {image_path}")
        return

    # Apply image processing functions
    processed_image_normalized = normalize_image(image)
    processed_image_reversed = reverse_image(image)
    processed_image_resized = resize_image(image)
    processed_image_sharpened = sharpen_edges(image)

    # Save the processed images in the same directory with appropriate names
    processed_base_name = os.path.splitext(os.path.basename(image_path))[0]

    cv2.imwrite(os.path.join(os.path.dirname(image_path), f"{processed_base_name}_normalized.jpg"), processed_image_normalized)
    cv2.imwrite(os.path.join(os.path.dirname(image_path), f"{processed_base_name}_reversed.jpg"), processed_image_reversed)
    cv2.imwrite(os.path.join(os.path.dirname(image_path), f"{processed_base_name}_resized.jpg"), processed_image_resized)
    cv2.imwrite(os.path.join(os.path.dirname(image_path), f"{processed_base_name}_sharpened.jpg"), processed_image_sharpened)

input_dir = r'D:\PENNY\Gaiat products\images'

for catalog in os.listdir(input_dir):
    catalog_path = os.path.join(input_dir, catalog)
    if not os.path.isdir(catalog_path):
        continue

    for category in os.listdir(catalog_path):
        category_path = os.path.join(catalog_path, category)
        if not os.path.isdir(category_path):
            continue

        # Use tqdm to create a progress bar for the product images in the category
        product_files = [os.path.join(category_path, f) for f in os.listdir(category_path) if f.endswith('.jpg')]
        progress_bar = tqdm(total=len(product_files), desc=f"Processing {catalog}/{category}", unit="image")

        # Use ThreadPoolExecutor for parallel processing
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(process_image, image_path) for image_path in product_files]

            for _ in futures:
                progress_bar.update(1)  # Update the progress bar for each processed image

        progress_bar.close()  # Close the progress bar for the current category


Processing Cleaning & Disinfections Chemicals/Air Fresheners: 100%|██████████| 8/8 [00:02<00:00,  3.57image/s]
Processing Cleaning & Disinfections Chemicals/Disinfections & General Cleaners: 100%|██████████| 11/11 [00:00<00:00, 73.12image/s]
Processing Cleaning & Disinfections Chemicals/Drain Opener: 100%|██████████| 12/12 [00:00<00:00, 27.19image/s]
Processing Cleaning & Disinfections Chemicals/Glass Cleaner: 100%|██████████| 3/3 [00:00<00:00, 22.89image/s]
Processing Cleaning & Disinfections Chemicals/Hand Soap: 100%|██████████| 8/8 [00:00<00:00, 16.56image/s]
Processing Cleaning & Disinfections Chemicals/Kitchen Cleaning Chemicals: 100%|██████████| 5/5 [00:00<00:00, 36.53image/s]
Processing Cleaning & Disinfections Chemicals/Specialized Chemicals: 100%|██████████| 25/25 [00:01<00:00, 20.92image/s]
Processing Cleaning & Disinfections Chemicals/Surface & Floor Cleaning: 100%|██████████| 5/5 [00:00<00:00, 12.58image/s]
Processing Cleaning & Disinfections Chemicals/Toilet Cleaning: 100%

Error: Unable to read image D:\PENNY\Gaiat products\images\Office items\Office stationery\حامل_ادوات_مكتبية.jpg


Processing Office items/Printers - Desktop: 100%|██████████| 4/4 [00:00<00:00, 29.31image/s]
Processing Office items/Software: 0image [00:00, ?image/s]
Processing Packaging Materials/Aluminum Containers: 0image [00:00, ?image/s]
Processing Packaging Materials/Paper Containers: 0image [00:00, ?image/s]
Processing Packaging Materials/Plastic Containers: 0image [00:00, ?image/s]
Processing Plastic, Tissues, & Paper Wares/Bags & Trash Bags: 100%|██████████| 7/7 [00:00<00:00, 12.93image/s]
Processing Plastic, Tissues, & Paper Wares/Paper Bags: 100%|██████████| 9/9 [00:00<00:00, 23.32image/s]
Processing Plastic, Tissues, & Paper Wares/Paper cups: 100%|██████████| 8/8 [00:00<00:00, 35.24image/s]
Processing Plastic, Tissues, & Paper Wares/Plastic Cups: 100%|██████████| 6/6 [00:00<00:00, 37.75image/s]
Processing Plastic, Tissues, & Paper Wares/Plastic Wares: 100%|██████████| 24/24 [00:01<00:00, 19.00image/s]
Processing Plastic, Tissues, & Paper Wares/Tissues: 100%|██████████| 15/15 [00:01<00:00

In [17]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

def normalize_image(image):
    return cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

def reverse_image(image):
    return cv2.bitwise_not(image)

def sharpen_edges(image):
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    return cv2.filter2D(image, -1, kernel)

def resize_image(image, new_size=(200, 200)):
    return cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)

def process_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to read image {image_path}")
        return

    # Apply image processing functions
    processed_image_normalized = normalize_image(image)
    processed_image_reversed = reverse_image(image)
    processed_image_resized = resize_image(image)
    processed_image_sharpened = sharpen_edges(image)

    # Save the processed images in the same directory with appropriate names
    processed_base_name = os.path.splitext(os.path.basename(image_path))[0]

    cv2.imwrite(os.path.join(os.path.dirname(image_path), f"{processed_base_name}_normalized.jpg"), processed_image_normalized)
    cv2.imwrite(os.path.join(os.path.dirname(image_path), f"{processed_base_name}_reversed.jpg"), processed_image_reversed)
    cv2.imwrite(os.path.join(os.path.dirname(image_path), f"{processed_base_name}_resized.jpg"), processed_image_resized)
    cv2.imwrite(os.path.join(os.path.dirname(image_path), f"{processed_base_name}_sharpened.jpg"), processed_image_sharpened)

input_dir = r'D:\PENNY\Gaiat products\images'

for root, _, files in os.walk(input_dir):
    # Use tqdm to create a progress bar for the images in each directory
    progress_bar = tqdm(total=len(files), desc=f"Processing {root}", unit="image")

    # Use ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_image, os.path.join(root, file)) for file in files]

        for _ in futures:
            progress_bar.update(1)  # Update the progress bar for each processed image

    progress_bar.close()  # Close the progress bar for the current directory


Processing D:\PENNY\Gaiat products\images: 0image [00:00, ?image/s]
Processing D:\PENNY\Gaiat products\images\Cleaning & Disinfections Chemicals: 0image [00:00, ?image/s]
Processing D:\PENNY\Gaiat products\images\Cleaning & Disinfections Chemicals\Air Fresheners: 100%|██████████| 69/69 [00:04<00:00, 16.87image/s]
Processing D:\PENNY\Gaiat products\images\Cleaning & Disinfections Chemicals\Disinfections & General Cleaners:   0%|          | 0/80 [00:00<?, ?image/s]

Error: Unable to read image D:\PENNY\Gaiat products\images\Cleaning & Disinfections Chemicals\Disinfections & General Cleaners\nebKblK_reversed.jpg
Error: Unable to read image D:\PENNY\Gaiat products\images\Cleaning & Disinfections Chemicals\Disinfections & General Cleaners\nebKblK_sharpened.jpg
Error: Unable to read image D:\PENNY\Gaiat products\images\Cleaning & Disinfections Chemicals\Disinfections & General Cleaners\V5ltJNm_normalized.jpg
Error: Unable to read image D:\PENNY\Gaiat products\images\Cleaning & Disinfections Chemicals\Disinfections & General Cleaners\V5ltJNm_resized.jpg
Error: Unable to read image D:\PENNY\Gaiat products\images\Cleaning & Disinfections Chemicals\Disinfections & General Cleaners\V5ltJNm_sharpened.jpg
Error: Unable to read image D:\PENNY\Gaiat products\images\Cleaning & Disinfections Chemicals\Disinfections & General Cleaners\yJMoyhZ_resized.jpg


Processing D:\PENNY\Gaiat products\images\Cleaning & Disinfections Chemicals\Disinfections & General Cleaners: 100%|██████████| 80/80 [00:01<00:00, 42.53image/s]
Processing D:\PENNY\Gaiat products\images\Cleaning & Disinfections Chemicals\Drain Opener: 100%|██████████| 61/61 [00:01<00:00, 44.58image/s]
Processing D:\PENNY\Gaiat products\images\Cleaning & Disinfections Chemicals\Glass Cleaner:   0%|          | 0/17 [00:00<?, ?image/s]

Error: Unable to read image D:\PENNY\Gaiat products\images\Cleaning & Disinfections Chemicals\Glass Cleaner\QNX5Ba2_resized.jpg
Error: Unable to read image D:\PENNY\Gaiat products\images\Cleaning & Disinfections Chemicals\Glass Cleaner\QNX5Ba2_sharpened.jpg


Processing D:\PENNY\Gaiat products\images\Cleaning & Disinfections Chemicals\Glass Cleaner: 100%|██████████| 17/17 [00:00<00:00, 45.33image/s]
Processing D:\PENNY\Gaiat products\images\Cleaning & Disinfections Chemicals\Hand Soap:   0%|          | 0/69 [00:00<?, ?image/s]

...

Processing D:\PENNY\Gaiat products\images\Stationery\Erasers & Sharpeners: 100%|██████████| 20/20 [00:00<00:00, 23.00image/s]
Processing D:\PENNY\Gaiat products\images\Stationery\Files:   0%|          | 0/130 [00:00<?, ?image/s]

Error: Unable to read image D:\PENNY\Gaiat products\images\Stationery\Files\118476_resized.jpg
Error: Unable to read image D:\PENNY\Gaiat products\images\Stationery\Files\119488_reversed.jpg
Error: Unable to read image D:\PENNY\Gaiat products\images\Stationery\Files\188913_reversed.jpg
Error: Unable to read image D:\PENNY\Gaiat products\images\Stationery\Files\218010_reversed.jpg
Error: Unable to read image D:\PENNY\Gaiat products\images\Stationery\Files\218011_resized.jpg
Error: Unable to read image D:\PENNY\Gaiat products\images\Stationery\Files\293100_normalized.jpg
Error: Unable to read image D:\PENNY\Gaiat products\images\Stationery\Files\293100_reversed.jpg


Processing D:\PENNY\Gaiat products\images\Stationery\Files: 100%|██████████| 130/130 [00:08<00:00, 14.67image/s]
Processing D:\PENNY\Gaiat products\images\Stationery\Filling Tools & Machines:   0%|          | 0/60 [00:00<?, ?image/s]

Error: Unable to read image D:\PENNY\Gaiat products\images\Stationery\Filling Tools & Machines\260130_normalized.jpg
Error: Unable to read image D:\PENNY\Gaiat products\images\Stationery\Filling Tools & Machines\65416_reversed.jpg
Error: Unable to read image D:\PENNY\Gaiat products\images\Stationery\Filling Tools & Machines\65421_normalized.jpg
Error: Unable to read image D:\PENNY\Gaiat products\images\Stationery\Filling Tools & Machines\65421_reversed.jpg
Error: Unable to read image D:\PENNY\Gaiat products\images\Stationery\Filling Tools & Machines\65421_resized.jpg
Error: Unable to read image D:\PENNY\Gaiat products\images\Stationery\Filling Tools & Machines\65421_sharpened.jpg


Processing D:\PENNY\Gaiat products\images\Stationery\Filling Tools & Machines: 100%|██████████| 60/60 [00:04<00:00, 13.36image/s]


In [18]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Pool
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, optimizers, utils

# Constants
input_dir = r'D:\PENNY\Gaiat products\images'
image_size = (200, 200)
batch_size = 32
epochs_parent = 10
epochs_category = 10
num_processes = 4  # Adjust the number of processes based on your CPU capacity

# Preprocess the images and load the data for parent classification
def preprocess_image_parent(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    image = cv2.resize(image, image_size)
    image = image.astype('float32') / 255.0  # Normalize to [0, 1]
    return image

def load_data_parent():
    images = []
    catalogs = []
    for root, _, files in os.walk(input_dir):
        catalog = os.path.basename(root)
        for file in files:
            if file.endswith('.jpg'):
                image_path = os.path.join(root, file)
                images.append(image_path)
                catalogs.append(catalog)

    return images, catalogs

def preprocess_and_predict_parent(image_path):
    image = preprocess_image_parent(image_path)
    return image, model_parent.predict(np.expand_dims(image, axis=0))

# Load the data for parent classification
images, catalogs = load_data_parent()


In [20]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Pool
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, optimizers, utils

# Constants
input_dir = r'D:\PENNY\Gaiat products\images'
image_size = (200, 200)
batch_size = 32
epochs_parent = 10
epochs_category = 10
num_processes = 4  # Adjust the number of processes based on your CPU capacity

# Preprocess the images and load the data for parent classification
def preprocess_image_parent(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    image = cv2.resize(image, image_size)
    image = image.astype('float32') / 255.0  # Normalize to [0, 1]
    return image

def load_data_parent():
    images = []
    catalogs = []
    for root, _, files in os.walk(input_dir):
        catalog = os.path.basename(root)
        for file in files:
            if file.endswith('.jpg'):
                image_path = os.path.join(root, file)
                images.append(image_path)
                catalogs.append(catalog)

    return images, catalogs

def preprocess_and_predict_parent(image_path):
    image = preprocess_image_parent(image_path)
    return image, model_parent.predict(np.expand_dims(image, axis=0))

# Load the data for parent classification
images, catalogs = load_data_parent()


In [ ]:
import tensorflow as tf


In [ ]:
# Build the CNN model for parent classification
model_parent = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(np.unique(catalogs)), activation='softmax')
])

# Compile the model for parent classification
model_parent.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Parallelize image processing for parent classification
with Pool(num_processes) as pool:
    results = list(tqdm(pool.imap(preprocess_and_predict_parent, images), total=len(images), desc="Processing Images", colour='pink'))

# Train the model for parent classification
model_parent.fit(np.array([result[0] for result in results]), np.array([result[1] for result in results]),
                 batch_size=batch_size, epochs=epochs_parent)


In [ ]:
# Preprocess the images and load the data for category classification
def preprocess_image_category(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    image = cv2.resize(image, image_size)
    image = image.astype('float32') / 255.0  # Normalize to [0, 1]
    return image

def load_data_category(images, catalogs):
    images_category = []
    categories = []

    for image_path, catalog in zip(images, catalogs):
        image = preprocess_image_category(image_path)
        images_category.append(image)
        categories.append(catalog_to_category[catalog])

    return np.array(images_category), np.array(categories)

# Prepare data for category classification using the predictions from the parent model
catalog_to_category = {catalog: idx for idx, catalog in enumerate(np.unique(catalogs))}
x_category, y_category = load_data_category([result[0] for result in results], catalogs)


In [ ]:
# Build the CNN model for category classification
model_category = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(np.unique(y_category)), activation='softmax')
])

# Compile the model for category classification
model_category.compile
